In [33]:
import pandas as pd 
import numpy as np
import os 
from dotenv import load_dotenv
load_dotenv()

os.chdir("..")


In [13]:
# define global constants 
chat_file = "./data/rnr_chat_w_transcript.csv"
agent_trans_file = "./data/agent_transcript_topics.csv"
customer_trans_file = "./data/customer_transcript_topics.csv"
survey_file = "./data/RnR PCS Survey 2021-10-08.csv"
product_file = "./data/RnR Product Reviews 2021-10-08.csv"
topic_file = "./data/Rocks_N_Ropes_Chat_2022-06-21v2.csv"
date_dim_file = "./data/date_dim.csv"

In [14]:
# read in data
chats = pd.read_csv(chat_file)
surveys = pd.read_csv(survey_file)
product_reviews = pd.read_csv(product_file) 
topics = pd.read_csv(topic_file)
agent_trans = pd.read_csv(agent_trans_file)
customer_trans = pd.read_csv(customer_trans_file)

# remove index becauase I forgot index = false
agent_trans = agent_trans.iloc[:,1:]
customer_trans = customer_trans.iloc[:,1:]
chats = chats.iloc[:,1:]


In [5]:
# join topics to transcripts 

agent_trans_ = agent_trans.merge(topics,how="left",left_on="topic_id_array", right_on="Id")
agent_trans_ = agent_trans_.assign(c_type = "a")
customer_trans_ = customer_trans.merge(topics,how="left",left_on="topic_id_array", right_on="Id")
customer_trans_ = customer_trans_.assign(c_type = "c")

trans_df =  pd.concat([agent_trans_, customer_trans_])

print(trans_df.shape)
trans_df.head()


(337531, 7)


,chat_id,topic_id_array,Id,Category,Topic,Keyword,c_type
0,5,838,838,Emotion,Delighted,great,a
1,5,2088,2088,Context,Time Measure,today,a
2,5,2217,2217,Context,Pricing-Positive,great price,a
3,5,2218,2218,Context,Pricing-Positive,a great price,a
4,5,1775,1775,Support,Communications,confirmation email,a


In [6]:
# join back to original chats 

chats_ = chats.merge(trans_df, how="left", left_on="chat_number", right_on="chat_id")


In [7]:
chats_.to_csv("./data/all_chats_merged.csv",index=False)

# Data Validation
- check NPS on filter 
     - is this on surveys ? 
- check Sentiment on filter

- check general volumes

In [8]:
dates = pd.read_csv(date_dim_file)
chats_ = chats_.merge(dates, how="left", left_on="m_contact_day_number", right_on="id")

In [9]:
chats_.to_csv("./data_all_chats_merged.csv",index=False)

In [10]:
surveys = surveys.merge(dates,how="left",left_on="TransactionDateUTC", right_on="id")
surveys = surveys.merge(dates,how="left",left_on="ResponseReceivedDateUTC", right_on="id")

In [11]:
surveys = surveys.rename(columns={
    "date_x":"TransactionDate",
    "date_y":"ResponseReceivedDate"
})

surveys = surveys.drop(columns=["id_x","id_y"])
surveys

,ContactRecordID,SurveyDetails,m_contact_record_ID,ContactName,ContactEmail,member_number,m_agent_ID,m_agent_name,m_agent_team_ID,survey_score,overall_experience_comment,TransactionDateUTC,ResponseReceivedDateUTC,TransactionDate,ResponseReceivedDate
0,782819,pcs_chat_v3,26934,SPENSE FOLKE,sfolke59@patch.com,NaN,141,Gratiana Moules,Campers,10,You value your customers,54,57,6/15/2022,6/18/2022
1,782820,pcs_chat_v3,1280,NERON SCRIVNER,nscrivneraj@sfgate.com,NaN,19,Johnathan Durbridge,Climbers,9,Always offers to help in extra ways,11,21,5/3/2022,5/13/2022
2,782821,pcs_chat_v3,10955,CLIFFORD RENEHAN,crenehanp1@mapquest.com,NaN,81,Myrtle Blachford,Campers,5,The code didn’t work two days later,18,18,5/10/2022,5/10/2022
3,782822,pcs_chat_v3,3379,BOYCEY SEAGER,bseagerg5@gmpg.org,NaN,173,Dorice Coffin,Runners,7,NaN,7,8,4/29/2022,4/30/2022
4,782823,pcs_chat_v3,11856,CORETTA BUZZA,cbuzzam2@tinypic.com,NaN,30,Abbey Rivilis,Han Solo,6,Not worth it,51,52,6/12/2022,6/13/2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028,784847,pcs_chat_v3,25727,GRIFFITH CICIOTTI,gciciottifv@ebay.com,NaN,42,Nevile Kubacki,Deep Water,5,I expected to receive my order a lot faster,34,35,5/26/2022,5/27/2022
2029,784848,pcs_chat_v3,27392,STANTON AKSELL,saksell8r@networkadvertising.org,503740.0,189,Aurie Larwood,NaN,5,It's frustrating that I have to chat you to ge...,23,26,5/15/2022,5/18/2022
2030,784849,pcs_chat_v3,29072,CHARMANE ZUPONE,czuponec4@geocities.com,NaN,135,Jeanne Everil,Climbers,4,I'm not happy with the shipping time for my order,25,25,5/17/2022,5/17/2022
2031,784850,pcs_chat_v3,29955,HILLEL SOFF,hsoffa3@amazon.co.jp,506220.0,193,Hermie Coppins,Runners,6,"You charge a premium for shipping, then use USPS?",31,31,5/23/2022,5/23/2022


In [13]:
surveys.to_csv("./data/surveys.csv",index=False)

In [14]:
for col in surveys.columns:
    surveys = surveys.rename(columns = {
        col: "s_" + col
    })

surveys.head()

,s_ContactRecordID,s_SurveyDetails,s_m_contact_record_ID,s_ContactName,s_ContactEmail,s_member_number,s_m_agent_ID,s_m_agent_name,s_m_agent_team_ID,s_survey_score,s_overall_experience_comment,s_TransactionDateUTC,s_ResponseReceivedDateUTC,s_TransactionDate,s_ResponseReceivedDate
0,782819,pcs_chat_v3,26934,SPENSE FOLKE,sfolke59@patch.com,NaN,141,Gratiana Moules,Campers,10,You value your customers,54,57,6/15/2022,6/18/2022
1,782820,pcs_chat_v3,1280,NERON SCRIVNER,nscrivneraj@sfgate.com,NaN,19,Johnathan Durbridge,Climbers,9,Always offers to help in extra ways,11,21,5/3/2022,5/13/2022
2,782821,pcs_chat_v3,10955,CLIFFORD RENEHAN,crenehanp1@mapquest.com,NaN,81,Myrtle Blachford,Campers,5,The code didn’t work two days later,18,18,5/10/2022,5/10/2022
3,782822,pcs_chat_v3,3379,BOYCEY SEAGER,bseagerg5@gmpg.org,NaN,173,Dorice Coffin,Runners,7,NaN,7,8,4/29/2022,4/30/2022
4,782823,pcs_chat_v3,11856,CORETTA BUZZA,cbuzzam2@tinypic.com,NaN,30,Abbey Rivilis,Han Solo,6,Not worth it,51,52,6/12/2022,6/13/2022


In [15]:
chats_ = chats_.merge(surveys, how="left", left_on="chat_number",right_on="s_m_contact_record_ID")

In [16]:
chats_.to_csv("./data/data_all_chats_merged.csv",index=False)

# Testing against CXI


In [1]:
import pandas as pd 
import numpy as np
import os 
from dotenv import load_dotenv
load_dotenv()

os.chdir("..")

In [8]:
all_data = pd.read_csv("./data/all_data_trunc.csv",sep=",")
surveys = pd.read_csv("./data/surveys.csv",sep=",")

In [15]:
topics.Topic.unique()

array(['Mad', 'Disapproving', 'Appreciative', 'Annoyed',
       'Dissatisfaction', 'Dismayed', 'Despair', 'Frustration',
       'Confusion', 'Resentful', 'Threats', 'Content', 'Loving',
       'Victimized', 'Awful', 'Disappointed', 'Disrespected', 'Delighted',
       'Indignant', 'Successful', 'Fraud-Transaction', 'Preferences',
       'Fit/Sizing-Other Issues', 'Shipping-Missing', 'Navigation',
       'Website-Cart', 'Defects/Integrity', 'No Sale',
       'Website-Browser Issues', 'Objection-No Interest',
       'Offer-Objection', 'Promo-Expiration', 'Shipping-Error',
       'Audio Issues', 'Lost or Stolen', 'Log-in-Password',
       'Objection-No Need', 'Password', 'Credit/Refund', 'Repairs',
       'Errors', 'Objection-Consider', 'Communication',
       'Fit/Sizing-Too Large', 'Objection-Refusal', 'Crashing',
       'Duplicate Payment', 'Dispute-Create', "Promo-Can't Apply",
       'Incorrect', 'Shipping-Damage', 'Payment', 'Warranty Coverage',
       'Payment Failure', 'Login Issue

In [3]:
filters = ["Errors", "Crashing", "Website Feedback-Negative", "Website-Broken Links/Pages"]
check_survey = all_data[
    all_data.Topic.isin(filters)
].copy()

In [21]:
check_survey["s_survey_score"].mean() # in python this means - but how does it agg in cxi? 


check_survey.site_down_sentiment.value_counts()

Negative    1848
Neutral      498
Name: site_down_sentiment, dtype: int64

In [11]:
surveys["survey_score"].unique()

array([10,  9,  5,  7,  6,  2,  8,  4,  1,  3,  0], dtype=int64)

In [ ]:
filters = ["Errors", "Crashing", "Website Feedback-Negative", "Website-Broken Links/Pages"]
check_survey = all_data[
    all_data.Topic.isin(filters)
].copy()

In [23]:
topics.Category.unique()

array(['Emotion', 'Issues', 'Product', 'Support', 'Context'], dtype=object)

# Check Promos Story

In [24]:
filters = ["Promo-Can't Apply","Promo-Exclusion"] # set as OR filter

promo = all_data[all_data.Topic.isin(filters)].copy()
promo.shape

(1027, 67)

In [29]:
blah = []
topics.Keyword.apply(lambda x : blah.append(x) if "women's" in x.lower() or "men's" in x.lower() or "size" in x.lower() else None)

0       None
1       None
2       None
3       None
4       None
        ... 
2989    None
2990    None
2991    None
2992    None
2993    None
Name: Keyword, Length: 2994, dtype: object

In [30]:
blah = [i for i in blah if not i is None]

In [31]:
blah

['incorrect size',
 'wrong size',
 'oversized',
 'smaller size',
 'larger size',
 "women's cycling shoes",
 "men's cycling shoes",
 'wear a size',
 'size chart',
 'what size would',
 'right size',
 'larger sizes',
 'true to size',
 'size',
 'best size',
 'larger sizes',
 "women's",
 "men's"]

In [32]:
surveys.overall_experience_comment

,ContactRecordID,SurveyDetails,m_contact_record_ID,ContactName,ContactEmail,member_number,m_agent_ID,m_agent_name,m_agent_team_ID,survey_score,overall_experience_comment,TransactionDateUTC,ResponseReceivedDateUTC
0,782819,pcs_chat_v3,26934,SPENSE FOLKE,sfolke59@patch.com,NaN,141,Gratiana Moules,Campers,10,You value your customers,54,57
1,782820,pcs_chat_v3,1280,NERON SCRIVNER,nscrivneraj@sfgate.com,NaN,19,Johnathan Durbridge,Climbers,9,Always offers to help in extra ways,11,21
2,782821,pcs_chat_v3,10955,CLIFFORD RENEHAN,crenehanp1@mapquest.com,NaN,81,Myrtle Blachford,Campers,5,The code didn’t work two days later,18,18
3,782822,pcs_chat_v3,3379,BOYCEY SEAGER,bseagerg5@gmpg.org,NaN,173,Dorice Coffin,Runners,7,NaN,7,8
4,782823,pcs_chat_v3,11856,CORETTA BUZZA,cbuzzam2@tinypic.com,NaN,30,Abbey Rivilis,Han Solo,6,Not worth it,51,52


# Testing google T5 for comment generation 

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer

In [11]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

In [35]:
input_ids = tokenizer.encode('summarize: steps to fix a tent zipper.', return_tensors='pt')
greedy_output = model.generate(input_ids, num_beams=7, no_repeat_ngram_size=1, min_length=50, max_length=1000)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
steps to fix a tent zipper. instructions for fixing zips are available on-line and in the store, or by calling 1-800-273-8255 (email: info@northwestdotcom).org/contact_french?lang=Fr&#x02BC;


In [ ]:
model.generate("Vests| gender | Men's && Gests | broken zipper")